# Comparing NBA and Euroleague Basketball
# Part 2: Data Cleaning

## Introduction

Basketball is one of the most popular sports in the world and it's easy to understand why: it involves a lot of scoring and its fast-paced tempo, paired with a rather straightforward set of rules, makes it accessible to a casual audience and exciting to watch. 

When talking about basketball, most people think about the NBA, the top league in the USA, which can count on some of the best players in the world. However, it is important to acknowledge that basketball is played professionally all around the world. Especially in Europe, this sport has deep solid roots and can count on a [long tradition](https://www.sports-fitness.co.uk/blog/growth-basketball-europe). In fact, many believe the level of competition there is comparable to the one of the NBA and, in [exhibition games](https://en.wikipedia.org/wiki/NBA_versus_EuroLeague_games#2010s) held during the off season, it's not unusual to see a European team beat an NBA squad.

The most important European basketball league is (very intuitively) called Euroleague. Unlike the NBA, where all franchises (except one) are American, teams from many different European countries participate in the Euroleague. Moreover, compared to the same 30 teams that are part of the NBA, only 16 to 18 different clubs compete in the Euroleague within a semi-open system; this means that, next to a slate of teams which return every year thanks to a special license, there are a few open slots assigned on merit according to the standings of each country's national league. Besides their format, the NBA and Euroleague also differ in some of their rules, from the size of the court to the actual in-game calls. The reader can refer to [this page](https://www.fiba.basketball/rule-differences) for an overview of these rule differences.   

---

In this notebook we clean the data we scraped in the first part of the project. The goal is to aggregate and uniformize the NBA and EL tables; in doing this, we will have to take into account the different playing time of games in the NBA and in the EL (this is one of the rules inconsistencies we mentioned above). At the end, we will have two DataFrames: one consisting of the players' data for both leagues and the other consisting of the teams' data for both leagues. 

## Cleaning Players' Data

### A First Look at the Data

Let's first work with the DataFrames containing the players' stats.

In [1]:
# Allow to run all code in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

# Read the CSV files containing the NBA and EL players' stats respectively
NBA_players = pd.read_csv("final_DataFrames/NBA/NBA_player_stats.csv")
EL_players = pd.read_csv("final_DataFrames/EL/EL_player_stats.csv")

In [3]:
# Allow display of all columns of a DataFrame
pd.set_option("display.max_columns", None)

# Take a look at both tables
NBA_players.head()
print("The shape of the NBA DataFrame is: {}".format(NBA_players.shape))
EL_players.head()
print("The shape of the NBA DataFrame is: {}".format(EL_players.shape))

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,0,1,Álex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,1,2,Quincy Acy,PF,26,TOT,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,2,2,Quincy Acy,PF,26,DAL,6,0,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2016,NBA
3,3,2,Quincy Acy,PF,26,BRK,32,1,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2016,NBA
4,4,3,Steven Adams,C,23,OKC,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


The shape of the NBA DataFrame is: (3323, 33)


,Unnamed: 0,#,Player,Team,GP,MPG,PPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year,League
0,0,1,Keith Langford,UNICS-Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,1,2,Nando De Colo,CSKA-Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,2,3,Andrew Goudelock,Maccabi-FOX-Tel-Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL
3,3,4,Brad Wanamaker,Darussafaka-Basketbol-Istanbul,34,33.5,16.7,5.3,11.8,0.448,1.6,4.3,0.386,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,2016,EL
4,4,5,Sergio Llull,Real-Madrid,33,27.8,16.5,5.4,12.9,0.416,2.2,6.7,0.330,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,2016,EL


The shape of the NBA DataFrame is: (1349, 26)


Since the NBA has more teams (hence more players) than the EL, the NBA table contains more data points. The first two columns of both DataFrames are meaningless so we can drop them.

In [4]:
# Drop first two columns of each DataFrame
NBA_players = NBA_players.iloc[:, 2:]
EL_players = EL_players.iloc[:, 2:]

In [5]:
# Take a look at the new tables
NBA_players.head()
print("The shape of the NBA DataFrame is: {}".format(NBA_players.shape))
EL_players.head()
print("The shape of the NBA DataFrame is: {}".format(EL_players.shape))

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,PF,26,TOT,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Quincy Acy,PF,26,DAL,6,0,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2016,NBA
3,Quincy Acy,PF,26,BRK,32,1,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2016,NBA
4,Steven Adams,C,23,OKC,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


The shape of the NBA DataFrame is: (3323, 31)


,Player,Team,GP,MPG,PPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year,League
0,Keith Langford,UNICS-Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,Nando De Colo,CSKA-Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,Andrew Goudelock,Maccabi-FOX-Tel-Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL
3,Brad Wanamaker,Darussafaka-Basketbol-Istanbul,34,33.5,16.7,5.3,11.8,0.448,1.6,4.3,0.386,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,2016,EL
4,Sergio Llull,Real-Madrid,33,27.8,16.5,5.4,12.9,0.416,2.2,6.7,0.330,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,2016,EL


The shape of the NBA DataFrame is: (1349, 24)


### Uniformizing Columns' Names

Among the remaining columns, a few of them are rather self-explanatory. In particular, the first column contains the players' names and the last two specify to what league and year each row belongs to. Perhaps, for those with little sports knowledge, it is harder to grasp the abbreviations used for the players' stats. Moreover, some of these stats only appear in the NBA DataFrame (which has seven extra columns) or they appear in both DataFrames but under different names. Therefore, for sake of clarity, let's explicitly go over the meaning of each column. We will list them as they are in the NBA table and then we will specify if they can also be found in the EL table (and under what name).   

`Pos`: player's playing position (it does not appear in the EL table)\
`Tm`: player's team (it appears in the EL table as `Team`)\
`G`: number of games played (it appears in the EL table as `GP`)\
`GS`: number of games started (it does not appear in the EL table)\
`MP`: minutes played (it appears in the EL table as `MPG`)\
`FG`, `FGA`, `FG%`: field goals made, attempted and field goals shooting percentage respectively (all these columns appear in the EL table but `FGM` is used rather than `FG`)\
`3P`, `3PA`,`3P%`: same as above but with 3-point shots instead\
`2P`, `2PA`,`2P%`: same as above but with 2-point shots instead (these columns do not appear in the EL table)\
`FT`, `FTA`,`FT%`: same as above but with free throw shots instead\
`eFG%`: effective field goal percentage (check [here](https://en.wikipedia.org/wiki/Effective_field_goal_percentage) for more on this stat; it does not appear in the EL table)\
`ORB`, `DRB`,`TRB`: offensive, defensive and total rebounds respectively (these columns appear in the EL table but `RPG` is used rather than `TRB`)\
`AST`: assists (it appears in the EL table as `APG`)\
`STL`: steals (it appears in the EL table as `SPG`)\
`BLK`: blockshots (it appears in the EL table as `BPG`)\
`TOV`: turnovers (it appears in the EL table with the same name)\
`PF`: personal fouls (it appears in the EL table with the same name)\
`PTS`: points (it appears in the EL table as `PPG`)

It is important to mention that all the stats above are averages per game. 

We will be able to obtain some of the information missing in the EL table using the data we have available. Yet, the `GS` column (representing the number of games started by a player) doesn't add much to our analysis so we get rid of it.

In [6]:
# Drop the `GS` column
NBA_players.drop(columns = "GS", inplace = True)

The first step to make the data more uniform is giving the columns the same names. This is what we do next using a combination of acronyms from both tables. 

In [7]:
# Uniformize the NBA_players columns' names
NBA_rename_dict = {"Tm" : "Team", "G" : "GP", "FG" : "FGM", "3P" : "3PM", "2P" : "2PM", "FT" : "FTM"}
NBA_players = NBA_players.rename(columns = NBA_rename_dict)

In [8]:
# Uniformize the EL_players columns' names
EL_rename_dict = {"MPG" : "MP", "RPG" : "TRB", "APG" : "AST", "SPG" : "STL", "BPG" : "BLK", "PPG" : "PTS"}
EL_players = EL_players.rename(columns = EL_rename_dict)

In [9]:
# Take a look at the new data
NBA_players.head()
EL_players.head()

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,SG,23,OKC,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,PF,26,TOT,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Quincy Acy,PF,26,DAL,6,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2016,NBA
3,Quincy Acy,PF,26,BRK,32,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2016,NBA
4,Steven Adams,C,23,OKC,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
0,Keith Langford,UNICS-Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,Nando De Colo,CSKA-Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,Andrew Goudelock,Maccabi-FOX-Tel-Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL
3,Brad Wanamaker,Darussafaka-Basketbol-Istanbul,34,33.5,16.7,5.3,11.8,0.448,1.6,4.3,0.386,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,2016,EL
4,Sergio Llull,Real-Madrid,33,27.8,16.5,5.4,12.9,0.416,2.2,6.7,0.330,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,2016,EL


### Uniformizing Teams' Names

Looking at the windows above, we notice that the teams' names are formatted in different ways depending on the table: the NBA DataFrame contains the teams' three letters nicknames while for the EL we have the teams' full names. As we did above, we want to uniformize this data. We do this by converting the NBA teams' nicknames with the help of a supporting table.     

In [10]:
# Read the CSV file containing the NBA teams' names and nicknames
NBA_nicknames = pd.read_csv("auxiliary_DataFrames/NBA_teams_nicknames.csv")

In [11]:
# Take a look at the data
NBA_nicknames

,Team,Nickname
0,Oklahoma City Thunder,OKC
1,Brooklyn Nets,BRK
2,Sacramento Kings,SAC
3,New Orleans Pelicans,NOP
4,Minnesota Timberwolves,MIN
5,San Antonio Spurs,SAS
6,Indiana Pacers,IND
7,Memphis Grizzlies,MEM
8,Portland Trail Blazers,POR
9,Cleveland Cavaliers,CLE


In [12]:
# Turn the DataFrame above into a dictionary with keys the team nicknames and value the team names
NBA_nicknames_dict = NBA_nicknames.set_index("Nickname")["Team"].to_dict()

# Rename the NBA teams in the players DataFrame
NBA_players["Team"] = NBA_players["Team"].replace(NBA_nicknames_dict)

In [13]:
# Look at the result
NBA_players.head()

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,PF,26,TOT,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Quincy Acy,PF,26,Dallas Mavericks,6,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2016,NBA
3,Quincy Acy,PF,26,Brooklyn Nets,32,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2016,NBA
4,Steven Adams,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


For the EL teams, we just replace the hyphens with spaces.

In [14]:
# Replace hyphens in EL teams' names
EL_players["Team"] = EL_players["Team"].str.replace("-", " ")

In [15]:
# Take a look at the final result
EL_players.head()

,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
0,Keith Langford,UNICS Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,Nando De Colo,CSKA Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,Andrew Goudelock,Maccabi FOX Tel Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL
3,Brad Wanamaker,Darussafaka Basketbol Istanbul,34,33.5,16.7,5.3,11.8,0.448,1.6,4.3,0.386,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,2016,EL
4,Sergio Llull,Real Madrid,33,27.8,16.5,5.4,12.9,0.416,2.2,6.7,0.330,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,2016,EL


### Dealing with Duplicates

It is important to realize that not all of the data we have available is relevant for the goal we have in mind. In fact, when we start to analyze and visualize the data in Part 3 of the project, we want to make sure to exclude marginal players who are not good representative of their league. One way to separate these marginal players is to set a minimum threshold involving games played and average minutes per game. 

Before being able to implement this strategy, however, there is a different issue we need to take care of: looking at the window of NBA players data above, we notice that some of the players appear more than once even within the same year. Let's expose these duplicates.

In [16]:
# Check NBA duplicates
NBA_players[NBA_players.duplicated(["Player", "Year"], keep = False)].head(6)

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
1,Quincy Acy,PF,26,TOT,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Quincy Acy,PF,26,Dallas Mavericks,6,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2016,NBA
3,Quincy Acy,PF,26,Brooklyn Nets,32,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2016,NBA
14,Justin Anderson,SF,23,TOT,75,16.4,2.5,5.9,0.424,0.8,2.6,0.299,1.7,3.3,0.522,0.490,1.4,1.7,0.791,0.9,2.3,3.3,0.9,0.5,0.3,0.9,1.5,7.1,2016,NBA
15,Justin Anderson,SF,23,Dallas Mavericks,51,13.9,2.2,5.4,0.401,0.7,2.4,0.303,1.5,3.0,0.477,0.468,1.4,1.7,0.795,0.8,2.2,2.9,0.6,0.5,0.3,0.8,1.3,6.5,2016,NBA
16,Justin Anderson,SF,23,Philadelphia 76ers,24,21.6,3.1,6.8,0.463,0.9,3.0,0.292,2.3,3.8,0.600,0.528,1.3,1.7,0.780,1.3,2.8,4.0,1.4,0.5,0.3,1.1,1.9,8.5,2016,NBA


It seems like the duplicates correspond to players who played for multiple teams within the same season. In particular, for these players we have separate rows for each one of the teams they played for, plus an extra row containing their total stats for the season. We recognize the latter rows because they all show the acronym `TOT` in the `Team` column.

For these players, we will keep their total stats and we will consider their team the one they played the most games for during the season.

In [17]:
# Create a mask to get the players who played for multiple teams within the same season
mask = NBA_players.duplicated(["Player", "Year"], keep = False)

# For each of these players look for the team they played most games for and assign the value to the Team column
NBA_players.loc[mask, "Team"] = NBA_players[mask].apply(lambda x :
                                                    NBA_players[(NBA_players["Player"] == x["Player"]) & 
                                                                (NBA_players["Year"] == x["Year"])].sort_values(
                                                        ["GP", "MP"], ascending = False).iloc[1]["Team"], axis = 1).copy()

In [18]:
# Drop the duplicates keeping only the first observation (the total stats row always apperead first)
NBA_players = NBA_players.drop_duplicates(["Player", "Year"], ignore_index = True)

In [19]:
# Make sure everything went right
NBA_players[NBA_players.duplicated(["Player", "Year"], keep = False)]
NBA_players[NBA_players["Team"] == "TOT"]

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League


,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League


In [20]:
# Take a look at the new data
NBA_players.head()

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA
3,Arron Afflalo,SG,31,Sacramento Kings,61,25.9,3.0,6.9,0.440,1.0,2.5,0.411,2.0,4.4,0.457,0.514,1.4,1.5,0.892,0.1,1.9,2.0,1.3,0.3,0.1,0.7,1.7,8.4,2016,NBA
4,Alexis Ajinça,C,28,New Orleans Pelicans,39,15.0,2.3,4.6,0.500,0.0,0.1,0.000,2.3,4.5,0.511,0.500,0.7,1.0,0.725,1.2,3.4,4.5,0.3,0.5,0.6,0.8,2.0,5.3,2016,NBA


Let's now check how the same source of duplicates is treated for the EL players.

In [21]:
# Look for EL duplicates
EL_duplicates = EL_players[EL_players.duplicated(["Player", "Year"], keep = False)]

EL_duplicates

,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
33,Alessandro Gentile,AX Armani Exchange Milan,9,22.9,10.8,4.2,9.2,0.458,0.3,1.6,0.214,2.0,3.6,0.562,1.3,1.7,3.0,2.4,1.1,0.3,2.2,2.1,2016,EL
158,Deon Thompson,KK Crvena Zvezda,13,10.5,4.3,1.8,3.2,0.561,0.0,0.0,0.000,0.8,1.5,0.500,0.6,0.8,1.5,0.5,0.3,0.3,0.2,1.2,2016,EL
159,Deon Thompson,Galatasaray,13,11.6,4.2,1.8,3.7,0.500,0.0,0.0,0.000,0.5,0.8,0.636,0.9,1.2,2.1,0.7,0.5,0.4,0.5,2.0,2016,EL
185,Alessandro Gentile,Panathinaikos,9,13.2,3.2,1.4,4.2,0.342,0.1,0.6,0.200,0.2,1.0,0.222,1.0,1.2,2.2,0.6,0.3,0.2,0.8,1.3,2016,EL
280,Dejan Musli,Brose Baskets Bamberg,15,19.8,10.4,4.0,6.7,0.594,0.0,0.0,0.000,2.4,3.5,0.679,2.0,4.1,6.1,1.4,0.3,0.3,1.4,1.7,2017,EL
386,Dejan Musli,Unicaja,9,11.2,5.4,2.1,3.7,0.576,0.0,0.0,0.000,1.2,2.2,0.550,0.8,1.9,2.7,0.4,0.0,0.3,0.9,0.7,2017,EL
571,Alen Omic,Buducnost,15,21.0,8.9,3.9,6.3,0.611,0.1,0.2,0.333,1.1,1.7,0.615,2.2,2.5,4.7,1.1,0.3,0.3,1.1,2.3,2018,EL
707,Alen Omic,AX Armani Exchange Milan,14,12.5,2.5,1.2,2.7,0.447,0.0,0.1,0.000,0.1,0.4,0.167,1.7,1.9,3.6,0.5,0.4,0.2,0.8,1.3,2018,EL
756,Kevin Punter,KK Crvena Zvezda,12,22.1,15.9,5.2,10.8,0.477,2.1,4.3,0.481,3.5,3.9,0.894,0.2,0.9,1.1,1.5,0.6,0.1,0.8,2.0,2019,EL
788,Vladimir Stimac,KK Crvena Zvezda,13,22.9,11.1,4.3,7.5,0.577,0.2,0.6,0.375,2.2,3.4,0.659,2.8,3.4,6.2,1.1,0.5,0.5,1.5,2.2,2019,EL


This time we don't have any extra row whith the players' total stats for the season; we'll have to create this rows ourselves.

In [22]:
def clean_duplicates(df):
    """
    Calculates the total stats of a player who played for multiple teams within the same season.
    The player's team will become the team he played most games for.
    
    :param df: the DataFrame containing the player's stats from each team he played for
    :type df: DataFrame
    
    :return: a Series containing the player's total average stats and updated team
    """
    
    # Initialize a total stats row
    total_stats = df.iloc[0].copy()
        
    # Update the Team column
    main_team = df.sort_values(["GP", "MP"], ascending = False).iloc[0]["Team"]
    total_stats["Team"] = main_team
    
    # Update the GP column
    total_games = df["GP"].sum()
    total_stats["GP"] = total_games
    
    # Calculate the player's cumulative stats for the season
    cumulative_stats = df.iloc[:, 3:-2].multiply(df["GP"], axis = "index").sum()
    # Calculate the player's average stats for the season
    average_stats = cumulative_stats / total_games
    
    # Update the stats columns (rounding to 3 decimal digits)
    total_stats.iloc[3:-2] = round(average_stats, 3)
    
    return total_stats

In [23]:
# Apply the function to the duplicates DataFrame
duplicates_tot_stats = EL_duplicates.groupby(["Player", "Year"]).apply(clean_duplicates)

# Take a look at the result
duplicates_tot_stats

,,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
Player,Year,,,,,,,,,,,,,,,,,,,,,,,,
Alen Omic,2018,Alen Omic,Buducnost,29,16.897,5.810,2.597,4.562,0.532,0.052,0.152,0.172,0.617,1.072,0.399,1.959,2.210,4.169,0.810,0.348,0.252,0.955,1.817,2018,EL
Alessandro Gentile,2016,Alessandro Gentile,AX Armani Exchange Milan,18,18.050,7.000,2.800,6.700,0.400,0.200,1.100,0.207,1.100,2.300,0.392,1.150,1.450,2.600,1.500,0.700,0.250,1.500,1.700,2016,EL
Dejan Musli,2017,Dejan Musli,Brose Baskets Bamberg,24,16.575,8.525,3.288,5.575,0.587,0.000,0.000,0.000,1.950,3.012,0.631,1.550,3.275,4.825,1.025,0.188,0.300,1.213,1.325,2017,EL
Deon Thompson,2016,Deon Thompson,Galatasaray,26,11.050,4.250,1.800,3.450,0.531,0.000,0.000,0.000,0.650,1.150,0.568,0.750,1.000,1.800,0.600,0.400,0.350,0.350,1.600,2016,EL
Kevin Punter,2019,Kevin Punter,KK Crvena Zvezda,23,20.330,11.548,3.909,9.557,0.392,1.383,3.917,0.337,2.448,2.896,0.801,0.248,1.235,1.483,1.165,0.552,0.100,0.704,2.048,2019,EL
Leo Westermann,2020,Leo Westermann,Barca,24,11.600,3.487,1.238,3.062,0.386,0.688,1.888,0.361,0.388,0.450,0.861,0.100,1.412,1.512,1.988,0.412,0.000,1.162,1.812,2020,EL
Vladimir Stimac,2019,Vladimir Stimac,KK Crvena Zvezda,21,18.024,8.624,3.348,5.595,0.624,0.124,0.371,0.232,1.781,2.714,0.672,2.038,2.790,4.790,0.833,0.310,0.310,1.310,1.933,2019,EL


Now that we have created these rows with the total stats, we can drop the duplicates in the EL DataFrame and replace them wth the clean data. To keep things consistent, we also make sure to round all the stats columns which do not refer to shooting percentages to 1 decimal digit.     

In [24]:
# Gather the columns to round
stats_cols = EL_players.columns[3:-2]
cols_to_round = [col for col in stats_cols if col not in ["FG%", "3P%", "FT%"]]

# Round the stats columns
duplicates_tot_stats[cols_to_round] = duplicates_tot_stats[cols_to_round].apply(lambda x : round(x, 1))

In [25]:
# Drop the duplicates for the EL players
to_drop = EL_duplicates.index
EL_players = EL_players.drop(index = to_drop)

In [26]:
# Add the total stats for the players we just dropped
EL_players = EL_players.append(duplicates_tot_stats, ignore_index = True)

In [27]:
# Make sure everything went right
EL_players[EL_players.duplicated(["Player", "Year"], keep = False)]
EL_players.tail(7)

,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League


,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
1335,Alen Omic,Buducnost,29,16.9,5.8,2.6,4.6,0.532,0.1,0.2,0.172,0.6,1.1,0.399,2.0,2.2,4.2,0.8,0.3,0.3,1.0,1.8,2018,EL
1336,Alessandro Gentile,AX Armani Exchange Milan,18,18.0,7.0,2.8,6.7,0.400,0.2,1.1,0.207,1.1,2.3,0.392,1.2,1.4,2.6,1.5,0.7,0.2,1.5,1.7,2016,EL
1337,Dejan Musli,Brose Baskets Bamberg,24,16.6,8.5,3.3,5.6,0.587,0.0,0.0,0.000,2.0,3.0,0.631,1.6,3.3,4.8,1.0,0.2,0.3,1.2,1.3,2017,EL
1338,Deon Thompson,Galatasaray,26,11.0,4.2,1.8,3.4,0.531,0.0,0.0,0.000,0.6,1.2,0.568,0.8,1.0,1.8,0.6,0.4,0.4,0.4,1.6,2016,EL
1339,Kevin Punter,KK Crvena Zvezda,23,20.3,11.5,3.9,9.6,0.392,1.4,3.9,0.337,2.4,2.9,0.801,0.2,1.2,1.5,1.2,0.6,0.1,0.7,2.0,2019,EL
1340,Leo Westermann,Barca,24,11.6,3.5,1.2,3.1,0.386,0.7,1.9,0.361,0.4,0.4,0.861,0.1,1.4,1.5,2.0,0.4,0.0,1.2,1.8,2020,EL
1341,Vladimir Stimac,KK Crvena Zvezda,21,18.0,8.6,3.3,5.6,0.624,0.1,0.4,0.232,1.8,2.7,0.672,2.0,2.8,4.8,0.8,0.3,0.3,1.3,1.9,2019,EL


### Filtering Out Irrelevant Data

We can now act on our plan of getting rid of marginal players with little to none playing time. It is tricky to establish what's the optimal treshold to separate these marginal players. After some thought, however, we decide to only keep players who have played at least 10 games and who have averaged more than 10 minutes per game.

In [28]:
# Keep only relevant data
NBA_players = NBA_players[(NBA_players["GP"] >=10) & (NBA_players["MP"] > 10)].reset_index(drop = True)
EL_players = EL_players[(EL_players["GP"] >=10) & (EL_players["MP"] > 10)].reset_index(drop = True)

In [29]:
# Check how much data is left
print("The shape of the remaining NBA data is: {}".format(NBA_players.shape))
print("The shape of the remaining EL data is: {}".format(EL_players.shape))

The shape of the remaining NBA data is: (2036, 30)
The shape of the remaining EL data is: (979, 24)


We still have a good amount of data left.

### Adding EL Players' Shooting Columns

As we have already observed, the NBA DataFrame has a few more columns than the EL one. Among these columns, four of them refer to specific shooting stats (`2P`, `2PA`, `2P%`, `eFG%`). Luckily, we can obtain this same information for the EL players using the data we have available. 

In [30]:
# Create the missing shooting columns for EL players matching the location in the NBA DataFrame
EL_players.insert(10, "2PM", EL_players["FGM"] - EL_players["3PM"])
EL_players.insert(11, "2PA", EL_players["FGA"] - EL_players["3PA"])
EL_players.insert(12, "2P%", EL_players["2PM"] / EL_players["2PA"])
EL_players.insert(13, "eFG%", (EL_players["FGM"] + 0.5*EL_players["3PM"]) / EL_players["FGA"])

In [31]:
# Take a look at the result
EL_players

,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,2PM,2PA,2P%,eFG%,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
0,Keith Langford,UNICS Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,4.9,11.2,0.437500,0.470238,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,Nando De Colo,CSKA Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,4.8,8.6,0.558140,0.577869,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,Andrew Goudelock,Maccabi FOX Tel Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,3.8,7.2,0.527778,0.604000,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL
3,Brad Wanamaker,Darussafaka Basketbol Istanbul,34,33.5,16.7,5.3,11.8,0.448,1.6,4.3,3.7,7.5,0.493333,0.516949,0.386,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,2016,EL
4,Sergio Llull,Real Madrid,33,27.8,16.5,5.4,12.9,0.416,2.2,6.7,3.2,6.2,0.516129,0.503876,0.330,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,2016,EL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,Dejan Musli,Brose Baskets Bamberg,24,16.6,8.5,3.3,5.6,0.587,0.0,0.0,3.3,5.6,0.589286,0.589286,0.000,2.0,3.0,0.631,1.6,3.3,4.8,1.0,0.2,0.3,1.2,1.3,2017,EL
975,Deon Thompson,Galatasaray,26,11.0,4.2,1.8,3.4,0.531,0.0,0.0,1.8,3.4,0.529412,0.529412,0.000,0.6,1.2,0.568,0.8,1.0,1.8,0.6,0.4,0.4,0.4,1.6,2016,EL
976,Kevin Punter,KK Crvena Zvezda,23,20.3,11.5,3.9,9.6,0.392,1.4,3.9,2.5,5.7,0.438596,0.479167,0.337,2.4,2.9,0.801,0.2,1.2,1.5,1.2,0.6,0.1,0.7,2.0,2019,EL
977,Leo Westermann,Barca,24,11.6,3.5,1.2,3.1,0.386,0.7,1.9,0.5,1.2,0.416667,0.500000,0.361,0.4,0.4,0.861,0.1,1.4,1.5,2.0,0.4,0.0,1.2,1.8,2020,EL


In [32]:
# Round the new shooting percentages to 3 decimal places
EL_players[["2P%", "eFG%"]] = EL_players[["2P%", "eFG%"]].round(3)

In [33]:
# Take a look at the two DataFrames
NBA_players.head(3)
print("The shape of the NBA DataFrame is: {}".format(NBA_players.shape))
EL_players.head(3)
print("The shape of the EL DataFrame is: {}".format(EL_players.shape))

,Player,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


The shape of the NBA DataFrame is: (2036, 30)


,Player,Team,GP,MP,PTS,FGM,FGA,FG%,3PM,3PA,2PM,2PA,2P%,eFG%,3P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,Year,League
0,Keith Langford,UNICS Kazan,28,34.0,21.8,6.9,16.8,0.409,2.0,5.6,4.9,11.2,0.438,0.470,0.363,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,2016,EL
1,Nando De Colo,CSKA Moscow,28,27.1,19.1,6.3,12.2,0.516,1.5,3.6,4.8,8.6,0.558,0.578,0.426,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,2016,EL
2,Andrew Goudelock,Maccabi FOX Tel Aviv,20,28.7,17.2,6.3,12.5,0.508,2.5,5.3,3.8,7.2,0.528,0.604,0.458,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,2016,EL


The shape of the EL DataFrame is: (979, 28)


### Adding Players' Biodata

The remaining extra columns in the NBA DataFrame are `Pos` and `Age`. The corresponding information for the EL players can be found in the rosters tables. 

In [34]:
# Read the CSV files containing the EL rosters
EL_rosters = pd.read_csv("final_DataFrames/EL/EL_rosters.csv")

EL_rosters.info()
EL_rosters.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597 entries, 0 to 1596
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1597 non-null   int64 
 1   #                 1589 non-null   object
 2   Player            1597 non-null   object
 3   Pos               1597 non-null   object
 4   Height            1597 non-null   object
 5   Weight            1597 non-null   object
 6   Age               1597 non-null   object
 7   Birth City        1545 non-null   object
 8   NBA Draft Status  1597 non-null   object
 9   Nationality       1597 non-null   object
 10  Year              1597 non-null   int64 
 11  League            1597 non-null   object
dtypes: int64(2), object(10)
memory usage: 149.8+ KB


,Unnamed: 0,#,Player,Pos,Height,Weight,Age,Birth City,NBA Draft Status,Nationality,Year,League
0,0,-,Danilo Andjusic,SG,6-4,200,25,Belgrade,"2013 NBA Draft, Undrafted",Serbia,2016,EL
1,1,2,Pavel Antipov,F,6-9,212,25,Tatarstan,"2013 NBA Draft, Undrafted",Russia,2016,EL
2,2,13,Marko Banic,SF,6-8,250,32,Zadar,"2006 NBA Draft, Undrafted",Croatia,2016,EL
3,3,4,Coty Clarke,F,6-7,235,24,Antioch (TN),"2014 NBA Draft, Undrafted",United States,2016,EL
4,4,12,Quim Colom,PG,6-2,194,28,Andorra La Vella,"2010 NBA Draft, Undrafted",AndorraSpain,2016,EL


In [35]:
# Read the CSV files containing the NBA rosters
NBA_rosters = pd.read_csv("final_DataFrames/NBA/NBA_rosters.csv")

NBA_rosters.info()
NBA_rosters.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2627 non-null   int64 
 1   No.         2627 non-null   object
 2   Player      2627 non-null   object
 3   Pos         2627 non-null   object
 4   Ht          2627 non-null   object
 5   Wt          2627 non-null   int64 
 6   Birth Date  2627 non-null   object
 7   Unnamed: 6  2627 non-null   object
 8   Exp         2627 non-null   object
 9   College     2216 non-null   object
 10  Year        2627 non-null   int64 
 11  League      2627 non-null   object
dtypes: int64(3), object(9)
memory usage: 246.4+ KB


,Unnamed: 0,No.,Player,Pos,Ht,Wt,Birth Date,Unnamed: 6,Exp,College,Year,League
0,0,8,Álex Abrines,SG,6-6,200,"August 1, 1993",es,R,NaN,2016,NBA
1,1,12,Steven Adams,C,6-11,265,"July 20, 1993",nz,3,Pitt,2016,NBA
2,2,6,Semaj Christon,PG,6-3,190,"November 1, 1992",us,R,Xavier,2016,NBA
3,3,30,Norris Cole,PG,6-2,175,"October 13, 1988",us,5,Cleveland State University,2016,NBA
4,4,4,Nick Collison,PF,6-10,255,"October 26, 1980",us,12,Kansas,2016,NBA


Besides the playing positions and ages of the EL players, we also have other interesting biographical data. In particular, we can use the players' height, weight and nationality. Before we add this information to the players' DataFrames, however, there are a couple of things we need to take care of. First of all let's uniformize the names of the columns we will keep.

In [36]:
# Rename the relevant columns in the NBA rosters table (the EL roster table is fine)
NBA_rosters.rename(columns = {"Ht": "Height", "Wt": "Weight", "Unnamed: 6" : "Nationality"}, inplace = True)

The players' heights in both rosters' tables are saved as strings; in fact, these heights are in American format and therefore an hyphen is used to separate feet and inches. Nervertheless, the `Age` and `Weight` columns of the EL rosters DataFrame are also of unexpectedly not numeric. Let's try to understand why that's the case.   

In [37]:
# Inspect unique values of `Weight` and `Age` columns for EL table
EL_rosters["Age"].sort_values().unique()
EL_rosters["Weight"].sort_values().unique()

array(['-', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40'], dtype=object)

array(['-', '152', '154', '156', '159', '160', '163', '165', '167', '169',
       '170', '172', '173', '174', '175', '176', '177', '178', '179',
       '180', '181', '182', '183', '184', '185', '186', '187', '188',
       '189', '190', '191', '192', '194', '195', '196', '197', '198',
       '200', '201', '202', '203', '204', '205', '206', '207', '208',
       '209', '210', '211', '212', '213', '214', '215', '216', '217',
       '218', '219', '220', '222', '223', '224', '225', '226', '227',
       '228', '229', '230', '231', '232', '233', '234', '235', '236',
       '237', '238', '239', '240', '241', '242', '243', '244', '245',
       '246', '247', '248', '250', '251', '252', '253', '254', '255',
       '256', '257', '260', '265', '267', '268', '270', '271', '275',
       '278', '290', '304'], dtype=object)

It seems like these columns have some `-` values which force all the other numbers to be stored as strings. These hyphens most likely denote missing values. Since these values might belong to irrelevant players who don't appear in the `EL_players` DataFrame anymore, we first do the merge operation to add the biodata and, in case it is necessary, we deal with any surviving missing value afterwards. 

In [38]:
# Add players' biodata to the `NBA_players` DataFrame
NBA_biodata_cols = ["Player", "Height", "Weight", "Nationality", "Year"]
NBA_players = NBA_players.merge(NBA_rosters[NBA_biodata_cols], how = "inner", on = ["Player", "Year"])

In [39]:
# Rearrange columns so that biodata comes first
cols = list(NBA_players.columns)

new_order_cols = [cols[0]]+cols[-3:]+cols[1:-3]
NBA_players = NBA_players[new_order_cols]

In [40]:
# Add players' biodata to the `EL_players` DataFrame
EL_biodata_cols = ["Player", "Pos", "Height", "Weight", "Age", "Nationality", "Year"]
EL_players = EL_players.merge(EL_rosters[EL_biodata_cols], how = "inner", on = ["Player", "Year"])

In [41]:
# Uniformize the order of the columns
EL_players = EL_players[NBA_players.columns]

In [42]:
# Take a look at the final result
NBA_players.head()
EL_players.head()

,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,6-6,200,es,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,6-7,240,us,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,6-11,265,nz,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA
3,Arron Afflalo,6-5,210,us,SG,31,Sacramento Kings,61,25.9,3.0,6.9,0.440,1.0,2.5,0.411,2.0,4.4,0.457,0.514,1.4,1.5,0.892,0.1,1.9,2.0,1.3,0.3,0.1,0.7,1.7,8.4,2016,NBA
4,Alexis Ajinça,7-2,248,fr,C,28,New Orleans Pelicans,39,15.0,2.3,4.6,0.500,0.0,0.1,0.000,2.3,4.5,0.511,0.500,0.7,1.0,0.725,1.2,3.4,4.5,0.3,0.5,0.6,0.8,2.0,5.3,2016,NBA


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Keith Langford,6-4,202,United States,G,33,UNICS Kazan,28,34.0,6.9,16.8,0.409,2.0,5.6,0.363,4.9,11.2,0.438,0.470,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,21.8,2016,EL
1,Nando De Colo,6-5,195,France,G,29,CSKA Moscow,28,27.1,6.3,12.2,0.516,1.5,3.6,0.426,4.8,8.6,0.558,0.578,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,19.1,2016,EL
2,Andrew Goudelock,6-3,200,United States,G,28,Maccabi FOX Tel Aviv,20,28.7,6.3,12.5,0.508,2.5,5.3,0.458,3.8,7.2,0.528,0.604,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,17.2,2016,EL
3,Brad Wanamaker,6-3,210,United States,SG,27,Darussafaka Basketbol Istanbul,34,33.5,5.3,11.8,0.448,1.6,4.3,0.386,3.7,7.5,0.493,0.517,4.5,5.2,0.864,0.5,2.6,3.1,4.7,1.5,0.1,3.0,2.8,16.7,2016,EL
4,Sergio Llull,6-3,190,Spain,PG,29,Real Madrid,33,27.8,5.4,12.9,0.416,2.2,6.7,0.330,3.2,6.2,0.516,0.504,3.5,4.2,0.847,0.3,1.5,1.8,5.9,0.7,0.1,2.3,1.9,16.5,2016,EL


We can now check if any of the `-` missing values survived the merge operation for the EL players table.

In [43]:
# Check for surviving missing values
for col in ["Weight", "Age"]:
    EL_players[EL_players[col] == "-"].shape[0]

0

0

All good: we can then go ahead and make the `Weight` and `Age` columns numeric.

In [44]:
# Change Dtype of `Weight` and `Age` columns
EL_players[["Weight", "Age"]] = EL_players[["Weight", "Age"]].astype("int64")

To make heights comparisons easier, let's also convert the `Height` column of both DataFrames to a numeric type by changing the units: from feet-inches to cm.

In [45]:
# Set up the correct conversion factors
fts_to_cm_factor = 30.48
inchs_to_cm_factor = 2.54

# Convert the `Height` column unit to cm
for df in [EL_players, NBA_players]:
    df["Height"] = df["Height"].apply(lambda x : round((int(x.split("-")[0]) * fts_to_cm_factor 
                                                               + int(x.split("-")[1]) * inchs_to_cm_factor))).copy()

Finally, in order to be consistent with the European metric system, let's also convert the weights unit: from pounds to kilos.

In [46]:
# Set up the correct conversion factors
lbs_to_kg_factor = 0.453

# Convert the `Weight` column unit to kg
for df in [NBA_players, EL_players]:
    df["Weight"] = df["Weight"].apply(lambda x : round(x * lbs_to_kg_factor))

In [47]:
# Take a look at the final result
NBA_players.head(3)
EL_players.head(3)

,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,es,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,201,109,us,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,211,120,nz,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Keith Langford,193,92,United States,G,33,UNICS Kazan,28,34.0,6.9,16.8,0.409,2.0,5.6,0.363,4.9,11.2,0.438,0.470,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,21.8,2016,EL
1,Nando De Colo,196,88,France,G,29,CSKA Moscow,28,27.1,6.3,12.2,0.516,1.5,3.6,0.426,4.8,8.6,0.558,0.578,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,19.1,2016,EL
2,Andrew Goudelock,190,91,United States,G,28,Maccabi FOX Tel Aviv,20,28.7,6.3,12.5,0.508,2.5,5.3,0.458,3.8,7.2,0.528,0.604,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,17.2,2016,EL


### Treating Missing Values

Let's see if the players' DataFrames have any missing values. We start with the NBA data.

In [48]:
# Count null values in each column
NBA_players.isnull().sum()

Player          0
Height          0
Weight          0
Nationality     0
Pos             0
Age             0
Team            0
GP              0
MP              0
FGM             0
FGA             0
FG%             0
3PM             0
3PA             0
3P%            65
2PM             0
2PA             0
2P%             0
eFG%            0
FTM             0
FTA             0
FT%             5
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
Year            0
League          0
dtype: int64

The only null values appear in the `3P%` and `FT%` columns. Let's take a closer look.

In [49]:
# Inspect NBA null values
NBA_players[NBA_players["3P%"].isnull()].head(3)
NBA_players[NBA_players["FT%"].isnull()].head(3)

,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
17,Ömer Aşık,213,116,tr,C,30,New Orleans Pelicans,31,15.5,1.0,2.1,0.477,0.0,0.0,NaN,1.0,2.1,0.477,0.477,0.7,1.3,0.590,1.5,3.7,5.3,0.5,0.2,0.3,0.5,1.6,2.7,2016,NBA
29,Aron Baynes,208,118,nz,C,30,Detroit Pistons,75,15.5,1.9,3.7,0.513,0.0,0.0,NaN,1.9,3.7,0.513,0.513,1.1,1.3,0.840,1.5,3.0,4.4,0.4,0.2,0.5,0.7,2.2,4.9,2016,NBA
38,Bismack Biyombo,203,116,cd,C,24,Orlando Magic,81,22.1,2.2,4.2,0.528,0.0,0.0,NaN,2.2,4.2,0.528,0.528,1.5,2.9,0.534,1.9,5.1,7.0,0.9,0.3,1.1,1.2,2.5,6.0,2016,NBA


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
50,Anthony Brown,201,95,us,SF,24,New Orleans Pelicans,11,14.5,1.6,4.5,0.360,0.6,2.5,0.259,1.0,2.1,0.478,0.430,0.0,0.0,NaN,0.7,2.3,3.0,0.7,0.5,0.1,0.5,1.4,3.9,2016,NBA
743,Mirza Teletović,206,110,ba,PF,32,Milwaukee Bucks,10,15.9,2.5,5.7,0.439,2.1,4.5,0.467,0.4,1.2,0.333,0.623,0.0,0.0,NaN,0.3,2.0,2.3,1.0,0.4,0.1,0.7,1.4,7.1,2017,NBA
772,James Webb III,206,92,us,SF,24,Brooklyn Nets,10,12.0,0.6,2.4,0.250,0.4,1.9,0.211,0.2,0.5,0.400,0.333,0.0,0.0,NaN,0.8,1.6,2.4,0.4,0.1,0.0,0.4,1.0,1.6,2017,NBA


It looks like null values in both the `3P%` and `FT%` columns are assigned when a player hasn't attempted any 3-point shot or any free throw respectively. This makes sense as the shooting percentage calculation for these players involves a division by zero. Let's explicitly confirm this observation.

In [50]:
# Further inspect null values
NBA_players.loc[NBA_players["3P%"].isnull(), "3PA"].sum()
NBA_players.loc[NBA_players["FT%"].isnull(), "FTA"].sum()

0.0

0.0

Yes, we were right. Depending on the role, it is not unusual for a basketball player not to shoot any 3-point shot or free throw; hence, dropping these rows would be too extreme. Instead, we replace the null values with zeros. In the future, in case we want to analyze the distribution of these columns, we will make sure to first filter out for players with at least one attempt.  

In [51]:
# Replace null values with 0
NBA_players[["3P%", "FT%"]] = NBA_players[["3P%", "FT%"]].fillna(0)

In [52]:
# Make sure there's no null value left
NBA_players.isnull().sum()

Player         0
Height         0
Weight         0
Nationality    0
Pos            0
Age            0
Team           0
GP             0
MP             0
FGM            0
FGA            0
FG%            0
3PM            0
3PA            0
3P%            0
2PM            0
2PA            0
2P%            0
eFG%           0
FTM            0
FTA            0
FT%            0
ORB            0
DRB            0
TRB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
Year           0
League         0
dtype: int64

Let's now look for any null values in the EL table.

In [53]:
# Count null values in each column
EL_players.isnull().sum()

Player         0
Height         0
Weight         0
Nationality    0
Pos            0
Age            0
Team           0
GP             0
MP             0
FGM            0
FGA            0
FG%            0
3PM            0
3PA            0
3P%            0
2PM            0
2PA            0
2P%            0
eFG%           0
FTM            0
FTA            0
FT%            0
ORB            0
DRB            0
TRB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
Year           0
League         0
dtype: int64

No missing values at all, that's great. However, considering what we have observed in the previous section, we should also make sure there's no missing value marked with an hyphen.

In [54]:
# Check for `-` missing values
null_values_dict = dict()

for col in EL_players.columns:
    null_values_dict[col] = EL_players[EL_players[col] == "-"].shape[0]
    
null_values_dict

{'Player': 0,
 'Height': 0,
 'Weight': 0,
 'Nationality': 0,
 'Pos': 0,
 'Age': 0,
 'Team': 0,
 'GP': 0,
 'MP': 0,
 'FGM': 0,
 'FGA': 0,
 'FG%': 0,
 '3PM': 0,
 '3PA': 0,
 '3P%': 0,
 '2PM': 0,
 '2PA': 0,
 '2P%': 0,
 'eFG%': 0,
 'FTM': 0,
 'FTA': 0,
 'FT%': 0,
 'ORB': 0,
 'DRB': 0,
 'TRB': 0,
 'AST': 0,
 'STL': 0,
 'BLK': 0,
 'TOV': 0,
 'PF': 0,
 'PTS': 0,
 'Year': 0,
 'League': 0}

Finally, let's check how the shooting percentages for players with no shot attempts are treated in the EL DataFrame.

In [55]:
# Check EL players with no shot attempts
EL_players[EL_players["FGA"] == 0].head(3)
EL_players[EL_players["3PA"] == 0].head(3)
EL_players[EL_players["2PA"] == 0].head(3)
EL_players[EL_players["FTA"] == 0].head(3)

,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
19,Ekpe Udoh,208,111,United StatesNigeria,PF,29,Fenerbahce Ulker,31,32.0,4.8,8.3,0.584,0.0,0.0,0.0,4.8,8.3,0.578,0.578,2.5,3.8,0.644,2.5,5.3,7.8,2.2,1.0,2.2,1.2,2.1,12.1,2016,EL
28,Artsiom Parakhouski,211,121,Belarus,C,29,UNICS Kazan,29,21.3,4.6,7.4,0.620,0.0,0.0,0.0,4.6,7.4,0.622,0.622,2.0,2.6,0.787,2.6,2.6,5.1,0.4,0.3,1.3,0.8,2.3,11.3,2016,EL
34,Bryant Dunston,203,106,United StatesArmenia,C,30,Anadolu Efes,35,29.0,4.0,7.3,0.551,0.0,0.0,0.0,4.0,7.3,0.548,0.548,2.7,3.9,0.679,2.9,3.3,6.2,1.1,0.6,1.1,0.9,2.7,10.7,2016,EL


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
164,Simone Fontecchio,201,91,Italy,SF,21,AX Armani Exchange Milan,10,11.7,1.3,3.0,0.433,0.7,1.7,0.412,0.6,1.3,0.462,0.550,0.0,0.0,0.0,0.6,0.9,1.5,0.8,0.5,0.1,0.5,2.1,3.3,2016,EL
356,Morayo Soluade,196,89,England,PG,22,Unicaja,11,10.4,0.9,2.7,0.333,0.4,1.6,0.222,0.5,1.1,0.455,0.407,0.0,0.0,0.0,0.1,0.6,0.7,1.4,0.6,0.3,1.1,1.1,2.2,2017,EL
540,Dogus Balbay,185,79,Turkey,G,30,Anadolu Efes,37,10.5,0.8,1.6,0.500,0.4,0.6,0.609,0.4,1.0,0.400,0.625,0.0,0.0,1.0,0.5,0.9,1.4,1.3,0.4,0.2,0.4,1.8,2.0,2018,EL


Regardless of the shot type, we observe 0 values for the shooting percentages. This is consistent with how we handled null values in the NBA table so we can keep going.

In [56]:
# Quick look at the current status of both DataFrames
NBA_players.head(3)
EL_players.head(3)

,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,es,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,201,109,us,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,211,120,nz,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


,Player,Height,Weight,Nationality,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Keith Langford,193,92,United States,G,33,UNICS Kazan,28,34.0,6.9,16.8,0.409,2.0,5.6,0.363,4.9,11.2,0.438,0.470,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,21.8,2016,EL
1,Nando De Colo,196,88,France,G,29,CSKA Moscow,28,27.1,6.3,12.2,0.516,1.5,3.6,0.426,4.8,8.6,0.558,0.578,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,19.1,2016,EL
2,Andrew Goudelock,190,91,United States,G,28,Maccabi FOX Tel Aviv,20,28.7,6.3,12.5,0.508,2.5,5.3,0.458,3.8,7.2,0.528,0.604,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,17.2,2016,EL


### Uniformzing the Geographical Data

Looking at both windows above, we notice that the `Nationality` column has different formats across the two DataFrames: in the NBA table nationalities are reported using the countries' [ISO 3166-1 alpha 2 code](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2) while the EL table contains the countries' full names. 

But that's not the only issue: inspecting the EL table more carefully, we notice that in some case players appear to have multiple nationalities.

In [57]:
# Check EL nationality values. Notice how some values include multiple countries.
EL_players["Nationality"].unique()

array(['United States', 'France', 'Spain', 'Serbia',
       'United StatesTurkey', 'Lithuania', 'United StatesMontenegro',
       'Greece', 'KosovoSerbia', 'United StatesNigeria', 'Latvia',
       'Italy', 'AndorraSpain', 'Belarus', 'Croatia', 'Australia',
       'Georgia', 'United StatesArmenia', 'Hungary', 'Turkey',
       'United StatesSlovenia', 'Finland', 'United StatesGeorgia',
       'Germany', 'United StatesIsrael', 'United StatesBelgium', 'Mexico',
       'United StatesGreece', 'Uruguay', 'Czech Republic',
       'Bosnia and HerzegovinaSerbia', 'United StatesCzech Republic',
       'United StatesAzerbaijan', 'Canada', 'CroatiaSerbia', 'Slovenia',
       'United StatesKosovo', 'CyprusBulgaria', 'SerbiaMontenegro',
       'Russia', 'North MacedoniaTurkey', 'United StatesAustria',
       'United StatesDominican Republic', 'United StatesSpain', 'Israel',
       'TurkeySlovenia', 'United StatesItaly', 'Brazil', 'Sweden',
       'North Macedonia', 'SpainSenegal',
       'United Stat

We can see how these double nationalities appear repeatadly and come in many different combinations. Seemingly, they are used to describe players with multiple international passports. Leaving this data as it is would make our analysis too fine and hard to read. Therefore, for players with multiple nationalities, we decide to only keep the first country that appears.

In [58]:
import re

# Use a regex to insert a comma after the first country in case of double nationality
EL_players["Nationality"] = EL_players["Nationality"].apply(lambda x : re.sub(r"(([A-Za-z]+\s*[A-Za-z]+)+(?=[A-Z]))",
                                                                              r"\1,", x))

# Split double nationalities and only keep the first one
EL_players["Nationality"] = EL_players["Nationality"].apply(lambda x : x.split(",")[0])

In [59]:
# Take a look at the newly formatted nations
EL_players["Nationality"].unique()

array(['United States', 'France', 'Spain', 'Serbia', 'Lithuania',
       'Greece', 'Kosovo', 'Latvia', 'Italy', 'Andorra', 'Belarus',
       'Croatia', 'Australia', 'Georgia', 'Hungary', 'Turkey', 'Finland',
       'Germany', 'Mexico', 'Uruguay', 'Czech Republic',
       'Bosnia and Herzegovina', 'Canada', 'Slovenia', 'Cyprus', 'Russia',
       'North Macedonia', 'Israel', 'Brazil', 'Sweden', 'Montenegro',
       'Poland', 'Argentina', 'Belgium', 'Cape Verde', 'England',
       'Nigeria', 'Estonia', 'Dominican Republic', 'Gabon', 'Iceland',
       'Cameroon', 'Republic of the Congo', 'Senegal', 'Cuba', 'Denmark',
       'Ghana'], dtype=object)

We still need to convert the two letter codes that identify nationalities in the NBA table to the countries full names. To help with this task, we use a very handy DataFrame we found on [Kaggle](https://www.kaggle.com/statchaitya/country-to-continent).

In [60]:
# We read the CSV file containing the countries codes and names
countries = pd.read_csv("auxiliary_DataFrames/country_codes.csv", encoding = "ISO-8859-1")

In [61]:
# Take a look at the data
countries

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248.0,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12.0,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16.0,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0
...,...,...,...,...,...,...,...,...,...
246,Wallis and Futuna,WF,WLF,876.0,ISO 3166-2:WF,Oceania,Polynesia,9.0,61.0
247,Western Sahara,EH,ESH,732.0,ISO 3166-2:EH,Africa,Northern Africa,2.0,15.0
248,Yemen,YE,YEM,887.0,ISO 3166-2:YE,Asia,Western Asia,142.0,145.0
249,Zambia,ZM,ZMB,894.0,ISO 3166-2:ZM,Africa,Eastern Africa,2.0,14.0


Besides the countries' codes and names, we also have the continent each country belongs to. This information can be helpful so it's worth keeping. All the other columns are not important so we get rid of them.

In [62]:
# Drop the columns we don't need
countries = countries.iloc[:, [0, 1, 5]]

# Rename the column with countries' codes
countries = countries.rename(columns = {"code_2" : "code"})

We can now use this DataFrame to properly format the countries' names in the NBA table. 

In [63]:
# Create a dictionary to replace the countries' codes
codes_dict = countries.set_index("code")["country"].to_dict()

# Replace the countries' codes with the countries' names
NBA_players["Nationality"] = NBA_players["Nationality"].str.upper().replace(codes_dict)

We also add a `Continent` column to both DataFrames.

In [64]:
# Create a dictionary to assign a continent to each country
continent_dict = countries.set_index("country")["continent"].to_dict()

# Create a `Continent` column in both DataFrames
for df in [NBA_players, EL_players]:
    continent_column = df["Nationality"].map(continent_dict)    
    df.insert(4, "Continent", continent_column)

In [65]:
# Make sure everything went right
NBA_players["Continent"].value_counts(dropna = False)
EL_players["Continent"].value_counts(dropna = False)

Americas    1658
Europe       254
Africa        58
Oceania       44
Asia          21
Name: Continent, dtype: int64

Europe      498
Americas    412
Asia         42
Africa       20
Oceania       7
Name: Continent, dtype: int64

In [66]:
# Final look at both DataFrames
NBA_players.head(3)
EL_players.head(3)

,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,Spain,Europe,SG,23,Oklahoma City Thunder,68,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2016,NBA
1,Quincy Acy,201,109,United States,Americas,PF,26,Brooklyn Nets,38,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2016,NBA
2,Steven Adams,211,120,New Zealand,Oceania,C,23,Oklahoma City Thunder,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2016,NBA


,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Keith Langford,193,92,United States,Americas,G,33,UNICS Kazan,28,34.0,6.9,16.8,0.409,2.0,5.6,0.363,4.9,11.2,0.438,0.470,6.0,7.2,0.832,0.7,2.7,3.4,3.7,0.8,0.2,2.4,1.8,21.8,2016,EL
1,Nando De Colo,196,88,France,Europe,G,29,CSKA Moscow,28,27.1,6.3,12.2,0.516,1.5,3.6,0.426,4.8,8.6,0.558,0.578,5.0,5.2,0.959,0.1,2.8,2.9,3.9,1.0,0.1,3.2,2.2,19.1,2016,EL
2,Andrew Goudelock,190,91,United States,Americas,G,28,Maccabi FOX Tel Aviv,20,28.7,6.3,12.5,0.508,2.5,5.3,0.458,3.8,7.2,0.528,0.604,2.1,2.3,0.913,0.2,2.5,2.6,2.9,0.2,0.0,1.7,1.6,17.2,2016,EL


### Normalizing the NBA and EL players' Data

We have succesfully cleaned and uniformized the tables containing the players' information for both leagues. Before combining the two DataFrames, however, we need to take into account one last crucial detail: in the NBA games last 48 minutes while in the EL only 40. Therefore, in order to be able to compare the data, we first need to normalize the stats from the two leagues.

There are a few standard ways of normalizing basketball stats in order to allow comparisons among players. A popular one suggests to consider [per-minute ratings](https://www.nbastuffer.com/analytics101/per-minute-ratings/#:~:text=Per%2Dminute%20ratings%20are%20calculated,he'd%20play%2036%20minutes.). This method is easy to understand but it has its own limitations since it tends to <a href="https://www.quora.com/Why-do-people-use-stats-per-game-over-stats-per-36-minutes-Isnt-that-more-accurate-for-example-for-a-player-only-played-for-2-or-6-minutes-every-game-5-times-in-a-row-thatll-hurt-his-ppg-but-in-reality-he-didnt-even#q-box">inflate the stats of players playing small number of minutes</a>. However, we have already filtered out these irrelevant players hence we go ahead and normalize our stats using the per 40-minutes system (we chose 40 minutes because that's how long games in the EL last).

In [67]:
# Collect the columns to normalize
cols_to_normalize = ["FGM", "FGA", "3PM", "3PA", "2PM", "2PA", "FTM", "FTA", "ORB", "DRB", "TRB",
                     "AST", "STL", "BLK","TOV", "PF", "PTS"]

# Normalize the players stats to per 40-minutes
NBA_players[cols_to_normalize] = NBA_players[cols_to_normalize].divide(NBA_players["MP"], axis = "index").multiply(40)
EL_players[cols_to_normalize] = EL_players[cols_to_normalize].divide(EL_players["MP"], axis = "index").multiply(40)

# Round the normalized stats to 1 decimal digit
NBA_players[cols_to_normalize] = NBA_players[cols_to_normalize].apply(lambda x : round(x, 1))
EL_players[cols_to_normalize] = EL_players[cols_to_normalize].apply(lambda x : round(x, 1))

In [68]:
# Recalculate the `eFG%` column
NBA_players["eFG%"] = (NBA_players["FGM"] + 0.5*NBA_players["3PM"]) / NBA_players["FGA"]
EL_players["eFG%"] = (EL_players["FGM"] + 0.5*EL_players["3PM"]) / EL_players["FGA"]

# Round this column to 3 decimal digits
NBA_players["eFG%"] = round(NBA_players["eFG%"], 3)
EL_players["eFG%"] = round(EL_players["eFG%"], 3)

In [69]:
# Take a look at the result
NBA_players.head(3)
EL_players.head(3)

,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,Spain,Europe,SG,23,Oklahoma City Thunder,68,15.5,5.2,12.9,0.393,3.6,9.3,0.381,1.5,3.6,0.426,0.543,1.5,1.8,0.898,0.8,2.6,3.4,1.5,1.3,0.3,1.3,4.4,15.5,2016,NBA
1,Quincy Acy,201,109,United States,Americas,PF,26,Brooklyn Nets,38,14.7,4.9,12.2,0.412,2.7,6.5,0.411,2.4,5.7,0.413,0.512,3.3,4.4,0.750,1.4,6.8,8.2,1.4,1.1,1.1,1.6,4.9,15.8,2016,NBA
2,Steven Adams,211,120,New Zealand,Oceania,C,23,Oklahoma City Thunder,80,29.9,6.3,11.0,0.571,0.0,0.0,0.000,6.3,11.0,0.572,0.573,2.7,4.3,0.611,4.7,5.6,10.3,1.5,1.5,1.3,2.4,3.2,15.1,2016,NBA


,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Keith Langford,193,92,United States,Americas,G,33,UNICS Kazan,28,34.0,8.1,19.8,0.409,2.4,6.6,0.363,5.8,13.2,0.438,0.470,7.1,8.5,0.832,0.8,3.2,4.0,4.4,0.9,0.2,2.8,2.1,25.6,2016,EL
1,Nando De Colo,196,88,France,Europe,G,29,CSKA Moscow,28,27.1,9.3,18.0,0.516,2.2,5.3,0.426,7.1,12.7,0.558,0.578,7.4,7.7,0.959,0.1,4.1,4.3,5.8,1.5,0.1,4.7,3.2,28.2,2016,EL
2,Andrew Goudelock,190,91,United States,Americas,G,28,Maccabi FOX Tel Aviv,20,28.7,8.8,17.4,0.508,3.5,7.4,0.458,5.3,10.0,0.528,0.606,2.9,3.2,0.913,0.3,3.5,3.6,4.0,0.3,0.0,2.4,2.2,24.0,2016,EL


### Combining the NBA and EL Players' DataFrames

We are now finally ready to combine the two tables and create a single DataFrame containing the information for players from both the NBA and the EL.

In [70]:
players = pd.concat([NBA_players, EL_players], ignore_index = True)

In [71]:
# Look at the final result
players.info()
players

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3014 entries, 0 to 3013
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       3014 non-null   object 
 1   Height       3014 non-null   int64  
 2   Weight       3014 non-null   int64  
 3   Nationality  3014 non-null   object 
 4   Continent    3014 non-null   object 
 5   Pos          3014 non-null   object 
 6   Age          3014 non-null   int64  
 7   Team         3014 non-null   object 
 8   GP           3014 non-null   int64  
 9   MP           3014 non-null   float64
 10  FGM          3014 non-null   float64
 11  FGA          3014 non-null   float64
 12  FG%          3014 non-null   float64
 13  3PM          3014 non-null   float64
 14  3PA          3014 non-null   float64
 15  3P%          3014 non-null   float64
 16  2PM          3014 non-null   float64
 17  2PA          3014 non-null   float64
 18  2P%          3014 non-null   float64
 19  eFG%  

,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,Spain,Europe,SG,23,Oklahoma City Thunder,68,15.5,5.2,12.9,0.393,3.6,9.3,0.381,1.5,3.6,0.426,0.543,1.5,1.8,0.898,0.8,2.6,3.4,1.5,1.3,0.3,1.3,4.4,15.5,2016,NBA
1,Quincy Acy,201,109,United States,Americas,PF,26,Brooklyn Nets,38,14.7,4.9,12.2,0.412,2.7,6.5,0.411,2.4,5.7,0.413,0.512,3.3,4.4,0.750,1.4,6.8,8.2,1.4,1.1,1.1,1.6,4.9,15.8,2016,NBA
2,Steven Adams,211,120,New Zealand,Oceania,C,23,Oklahoma City Thunder,80,29.9,6.3,11.0,0.571,0.0,0.0,0.000,6.3,11.0,0.572,0.573,2.7,4.3,0.611,4.7,5.6,10.3,1.5,1.5,1.3,2.4,3.2,15.1,2016,NBA
3,Arron Afflalo,196,95,United States,Americas,SG,31,Sacramento Kings,61,25.9,4.6,10.7,0.440,1.5,3.9,0.411,3.1,6.8,0.457,0.500,2.2,2.3,0.892,0.2,2.9,3.1,2.0,0.5,0.2,1.1,2.6,13.0,2016,NBA
4,Alexis Ajinça,218,112,France,Europe,C,28,New Orleans Pelicans,39,15.0,6.1,12.3,0.500,0.0,0.3,0.000,6.1,12.0,0.511,0.496,1.9,2.7,0.725,3.2,9.1,12.0,0.8,1.3,1.6,2.1,5.3,14.1,2016,NBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009,Dejan Musli,213,110,Serbia,Europe,C,27,Brose Baskets Bamberg,24,16.6,8.0,13.5,0.587,0.0,0.0,0.000,8.0,13.5,0.589,0.593,4.8,7.2,0.631,3.9,8.0,11.6,2.4,0.5,0.7,2.9,3.1,20.5,2017,EL
3010,Deon Thompson,203,113,United States,Americas,PF,28,Galatasaray,26,11.0,6.5,12.4,0.531,0.0,0.0,0.000,6.5,12.4,0.529,0.524,2.2,4.4,0.568,2.9,3.6,6.5,2.2,1.5,1.5,1.5,5.8,15.3,2016,EL
3011,Kevin Punter,188,86,United States,Americas,SG,26,KK Crvena Zvezda,23,20.3,7.7,18.9,0.392,2.8,7.7,0.337,4.9,11.2,0.439,0.481,4.7,5.7,0.801,0.4,2.4,3.0,2.4,1.2,0.2,1.4,3.9,22.7,2019,EL
3012,Leo Westermann,198,89,France,Europe,SF,28,Barca,24,11.6,4.1,10.7,0.386,2.4,6.6,0.361,1.7,4.1,0.417,0.495,1.4,1.4,0.861,0.3,4.8,5.2,6.9,1.4,0.0,4.1,6.2,12.1,2020,EL


In [72]:
# Change "United States" to "USA" for ease of read
players = players.replace("United States", "USA") 

In [73]:
# Take a new look
players

,Player,Height,Weight,Nationality,Continent,Pos,Age,Team,GP,MP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Álex Abrines,198,91,Spain,Europe,SG,23,Oklahoma City Thunder,68,15.5,5.2,12.9,0.393,3.6,9.3,0.381,1.5,3.6,0.426,0.543,1.5,1.8,0.898,0.8,2.6,3.4,1.5,1.3,0.3,1.3,4.4,15.5,2016,NBA
1,Quincy Acy,201,109,USA,Americas,PF,26,Brooklyn Nets,38,14.7,4.9,12.2,0.412,2.7,6.5,0.411,2.4,5.7,0.413,0.512,3.3,4.4,0.750,1.4,6.8,8.2,1.4,1.1,1.1,1.6,4.9,15.8,2016,NBA
2,Steven Adams,211,120,New Zealand,Oceania,C,23,Oklahoma City Thunder,80,29.9,6.3,11.0,0.571,0.0,0.0,0.000,6.3,11.0,0.572,0.573,2.7,4.3,0.611,4.7,5.6,10.3,1.5,1.5,1.3,2.4,3.2,15.1,2016,NBA
3,Arron Afflalo,196,95,USA,Americas,SG,31,Sacramento Kings,61,25.9,4.6,10.7,0.440,1.5,3.9,0.411,3.1,6.8,0.457,0.500,2.2,2.3,0.892,0.2,2.9,3.1,2.0,0.5,0.2,1.1,2.6,13.0,2016,NBA
4,Alexis Ajinça,218,112,France,Europe,C,28,New Orleans Pelicans,39,15.0,6.1,12.3,0.500,0.0,0.3,0.000,6.1,12.0,0.511,0.496,1.9,2.7,0.725,3.2,9.1,12.0,0.8,1.3,1.6,2.1,5.3,14.1,2016,NBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009,Dejan Musli,213,110,Serbia,Europe,C,27,Brose Baskets Bamberg,24,16.6,8.0,13.5,0.587,0.0,0.0,0.000,8.0,13.5,0.589,0.593,4.8,7.2,0.631,3.9,8.0,11.6,2.4,0.5,0.7,2.9,3.1,20.5,2017,EL
3010,Deon Thompson,203,113,USA,Americas,PF,28,Galatasaray,26,11.0,6.5,12.4,0.531,0.0,0.0,0.000,6.5,12.4,0.529,0.524,2.2,4.4,0.568,2.9,3.6,6.5,2.2,1.5,1.5,1.5,5.8,15.3,2016,EL
3011,Kevin Punter,188,86,USA,Americas,SG,26,KK Crvena Zvezda,23,20.3,7.7,18.9,0.392,2.8,7.7,0.337,4.9,11.2,0.439,0.481,4.7,5.7,0.801,0.4,2.4,3.0,2.4,1.2,0.2,1.4,3.9,22.7,2019,EL
3012,Leo Westermann,198,89,France,Europe,SF,28,Barca,24,11.6,4.1,10.7,0.386,2.4,6.6,0.361,1.7,4.1,0.417,0.495,1.4,1.4,0.861,0.3,4.8,5.2,6.9,1.4,0.0,4.1,6.2,12.1,2020,EL


In [74]:
# Save the DataFrame to a CSV file
players.to_csv("final_DataFrames/player_stats_clean.csv")

## Cleaning Teams' Data

### A First Look at the Data

Let's now take care of the DataFrames containing the teams' data. 

In [75]:
# Read the CSV files containing the NBA and EL teams' stats respectively
NBA_teams = pd.read_csv("final_DataFrames/NBA/NBA_team_stats.csv")
EL_teams = pd.read_csv("final_DataFrames/EL/EL_team_stats.csv")

In [76]:
# Take a look at the data
NBA_teams.head()
EL_teams.head()

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,0,1,Golden State Warriors*,82,241.2,43.1,87.1,0.495,12.0,31.2,0.383,31.1,55.8,0.557,17.8,22.6,0.788,9.4,35.0,44.4,30.4,9.6,6.8,14.8,19.3,115.9,2016,NBA
1,1,2,Houston Rockets*,82,241.2,40.3,87.2,0.462,14.4,40.3,0.357,25.9,46.9,0.552,20.3,26.5,0.766,10.9,33.5,44.4,25.2,8.2,4.3,15.1,19.9,115.3,2016,NBA
2,2,3,Denver Nuggets,82,240.9,41.2,87.7,0.469,10.6,28.8,0.368,30.6,58.9,0.519,18.7,24.2,0.774,11.8,34.6,46.4,25.3,6.9,3.9,15.0,19.1,111.7,2016,NBA
3,3,4,Cleveland Cavaliers*,82,242.4,39.9,84.9,0.470,13.0,33.9,0.384,26.9,51.0,0.528,17.5,23.3,0.748,9.3,34.4,43.7,22.7,6.6,4.0,13.7,18.1,110.3,2016,NBA
4,4,5,Washington Wizards*,82,242.1,41.3,87.0,0.475,9.2,24.8,0.372,32.1,62.3,0.515,17.3,22.1,0.784,10.3,32.6,42.9,23.9,8.5,4.1,14.2,21.3,109.2,2016,NBA


,Unnamed: 0,#,Team,GP,MPG,PPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year,League
0,0,1,CSKA Moscow,35,40.3,87.5,30.0,60.2,0.498,8.7,21.6,0.403,18.8,22.9,0.821,7.8,22.3,30.1,19.9,7.1,3.2,14.1,22.6,2016,EL
1,1,2,Real Madrid,36,40.0,85.3,30.8,63.4,0.485,9.7,26.1,0.373,14.0,18.2,0.768,9.4,23.6,33.0,20.2,6.7,2.9,11.9,21.1,2016,EL
2,2,3,Anadolu Efes,35,40.6,82.0,30.0,67.3,0.445,7.6,22.5,0.337,14.4,19.7,0.734,10.6,21.6,32.2,17.9,6.9,3.4,11.3,19.4,2016,EL
3,3,4,Baskonia,33,40.0,82.0,30.0,64.0,0.468,8.2,23.1,0.354,13.8,18.2,0.763,9.3,23.5,32.8,18.4,6.7,3.0,13.2,21.6,2016,EL
4,4,5,AX Armani Exchange Milan,30,40.2,80.7,29.5,63.0,0.469,7.1,20.2,0.350,14.7,19.5,0.753,9.4,20.4,29.8,17.4,7.0,1.5,14.0,22.0,2016,EL


The columns of both DataFrames are analogous to the ones that appear in the corresponding players' tables. Hence, we can just repeat the same cleaning operations using the same variables we used before. This time, however, we will not keep the `MP` column since this statistic applied to teams is not really meaningful. 

In [77]:
# Rename columns the same way we did for the players DataFrames 
NBA_teams.rename(columns = NBA_rename_dict, inplace = True)
EL_teams.rename(columns = EL_rename_dict, inplace = True)

In [78]:
# Create the missing shooting columns in EL_teams matching the location in NBA_teams
EL_teams.insert(11, "2PM", EL_teams["FGM"] - EL_teams["3PM"])
EL_teams.insert(12, "2PA", EL_teams["FGA"] - EL_teams["3PA"])
EL_teams.insert(13, "2P%", EL_teams["2PM"] / EL_teams["2PA"])

In [79]:
# Round the new percentage column we added to 3 decimal digits
EL_teams["2P%"] = round(EL_teams["2P%"], 3)

In [80]:
# Rearrange columns matching the stats columns in the players DataFrames (except MP and "eFG%")
cols = ["Team", "GP"] + list(players.columns)[10:]
cols.remove("eFG%")

NBA_teams = NBA_teams[cols]
EL_teams = EL_teams[cols]

In [81]:
# Take a look at the result
NBA_teams.head(3)
EL_teams.head(3)

,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,Golden State Warriors*,82,43.1,87.1,0.495,12.0,31.2,0.383,31.1,55.8,0.557,17.8,22.6,0.788,9.4,35.0,44.4,30.4,9.6,6.8,14.8,19.3,115.9,2016,NBA
1,Houston Rockets*,82,40.3,87.2,0.462,14.4,40.3,0.357,25.9,46.9,0.552,20.3,26.5,0.766,10.9,33.5,44.4,25.2,8.2,4.3,15.1,19.9,115.3,2016,NBA
2,Denver Nuggets,82,41.2,87.7,0.469,10.6,28.8,0.368,30.6,58.9,0.519,18.7,24.2,0.774,11.8,34.6,46.4,25.3,6.9,3.9,15.0,19.1,111.7,2016,NBA


,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League
0,CSKA Moscow,35,30.0,60.2,0.498,8.7,21.6,0.403,21.3,38.6,0.552,18.8,22.9,0.821,7.8,22.3,30.1,19.9,7.1,3.2,14.1,22.6,87.5,2016,EL
1,Real Madrid,36,30.8,63.4,0.485,9.7,26.1,0.373,21.1,37.3,0.566,14.0,18.2,0.768,9.4,23.6,33.0,20.2,6.7,2.9,11.9,21.1,85.3,2016,EL
2,Anadolu Efes,35,30.0,67.3,0.445,7.6,22.5,0.337,22.4,44.8,0.500,14.4,19.7,0.734,10.6,21.6,32.2,17.9,6.9,3.4,11.3,19.4,82.0,2016,EL


Lastly, we uniformize the teams' names to match the ones in the `players` DataFrame. In particular, we remove the asterisks appearing next to some of the NBA teams' names and we also get rid of any eventual hyphen in the EL teams' names.

In [82]:
# Remove * from NBA team names
NBA_teams["Team"] = NBA_teams["Team"].str.replace("*", "", regex = False) 

# Remove - from EL team names
EL_teams["Team"] = EL_teams["Team"].str.replace("-", " ", regex = False) 

### Adding Teams' Winning Percentages

The last piece of information we want to add to the teams DataFrames is the winning percentages. This information is contained in two other tables we scraped from the web.

In [83]:
# Read the CSV files containing the NBA and EL teams' records respectively
NBA_records = pd.read_csv("final_DataFrames/NBA/NBA_team_records.csv")
EL_records = pd.read_csv("final_DataFrames/EL/EL_team_records.csv")

In [84]:
# Have a look at the data
NBA_records.head(3)
EL_records.head(3)

,Unnamed: 0,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,League
0,0,Philadelphia 76ers*,49,23,0.681,—,113.6,108.1,5.28,2016,NBA
1,1,Brooklyn Nets*,48,24,0.667,1.0,118.6,114.1,4.24,2016,NBA
2,2,Milwaukee Bucks*,46,26,0.639,3.0,120.1,114.2,5.57,2016,NBA


,Unnamed: 0,#,Team,W,L,PCT,GB,L10,STRK,PPG,OPPG,DIFF,Home,Away,Year,League
0,0,1,Real Madrid,23,7,0.767,0,7-3,L 2,86.2,78.4,7.8,14-1,9-6,2016,EL
1,1,2,CSKA Moscow,22,8,0.733,1,7-3,W 1,87.3,79.6,7.7,14-1,8-7,2016,EL
2,2,3,Olympiacos,19,11,0.633,4,4-6,L 1,77.9,74.2,3.7,11-4,8-7,2016,EL


Once again we need to make sure the teams' names match the ones in the other tables. Moreover, we also uniformize the name of the column containing the winning percentages.

In [85]:
# Remove the asterisk from the NBA team names
NBA_records["Team"] = NBA_records["Team"].str.replace("*", "", regex = False)

# Remove the hyphen from the EL team names
EL_records["Team"] = EL_records["Team"].str.replace("-", " ", regex = False)

In [86]:
# Uniformize the name of the column containing the teams' winning percentages
EL_records = EL_records.rename(columns = {"PCT" : "W/L%"})

Now we are ready to merge the data.

In [87]:
# Add the winning percentages to the teams tables
NBA_teams_clean = NBA_teams.merge(NBA_records[["Team", "W/L%", "Year"]], on = ["Team", "Year"], how = "left")
EL_teams_clean = EL_teams.merge(EL_records[["Team", "W/L%", "Year"]], on = ["Team", "Year"], how = "left")

In [88]:
# Look at the final result
NBA_teams_clean.head(3)
EL_teams_clean.head(3)

,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League,W/L%
0,Golden State Warriors,82,43.1,87.1,0.495,12.0,31.2,0.383,31.1,55.8,0.557,17.8,22.6,0.788,9.4,35.0,44.4,30.4,9.6,6.8,14.8,19.3,115.9,2016,NBA,0.542
1,Houston Rockets,82,40.3,87.2,0.462,14.4,40.3,0.357,25.9,46.9,0.552,20.3,26.5,0.766,10.9,33.5,44.4,25.2,8.2,4.3,15.1,19.9,115.3,2016,NBA,0.236
2,Denver Nuggets,82,41.2,87.7,0.469,10.6,28.8,0.368,30.6,58.9,0.519,18.7,24.2,0.774,11.8,34.6,46.4,25.3,6.9,3.9,15.0,19.1,111.7,2016,NBA,0.653


,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League,W/L%
0,CSKA Moscow,35,30.0,60.2,0.498,8.7,21.6,0.403,21.3,38.6,0.552,18.8,22.9,0.821,7.8,22.3,30.1,19.9,7.1,3.2,14.1,22.6,87.5,2016,EL,0.733
1,Real Madrid,36,30.8,63.4,0.485,9.7,26.1,0.373,21.1,37.3,0.566,14.0,18.2,0.768,9.4,23.6,33.0,20.2,6.7,2.9,11.9,21.1,85.3,2016,EL,0.767
2,Anadolu Efes,35,30.0,67.3,0.445,7.6,22.5,0.337,22.4,44.8,0.500,14.4,19.7,0.734,10.6,21.6,32.2,17.9,6.9,3.4,11.3,19.4,82.0,2016,EL,0.567


### Normalizing the NBA and EL Teams' Data

Before combining the two DataFrames into a single table, we need to normalize the stats using the same per 40-minutes system we adopted for the players' data. Since 40 minutes is also the duration of a game in the EL, we only need to operate on the NBA table.

In [89]:
# Collect the columns to normalize
stat_cols = list(NBA_teams.columns)[2:-2]
cols_to_normalize = [col for col in stat_cols if col not in ["FG%", "3P%", "2P%", "FT%"]] 

In [90]:
# Initialize the normalizing factor
NBA_to_EL_norm_factor = 40/48

# Normalize NBA team stats to per 40-minutes 
NBA_teams_clean[cols_to_normalize] = NBA_teams_clean[cols_to_normalize] * NBA_to_EL_norm_factor

# Round the normalized stats to 1 decimal digit
NBA_teams_clean[cols_to_normalize] = NBA_teams_clean[cols_to_normalize].apply(lambda x : round(x, 1))

In [91]:
# Take a look at the result
NBA_teams_clean.head(3)

,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League,W/L%
0,Golden State Warriors,82,35.9,72.6,0.495,10.0,26.0,0.383,25.9,46.5,0.557,14.8,18.8,0.788,7.8,29.2,37.0,25.3,8.0,5.7,12.3,16.1,96.6,2016,NBA,0.542
1,Houston Rockets,82,33.6,72.7,0.462,12.0,33.6,0.357,21.6,39.1,0.552,16.9,22.1,0.766,9.1,27.9,37.0,21.0,6.8,3.6,12.6,16.6,96.1,2016,NBA,0.236
2,Denver Nuggets,82,34.3,73.1,0.469,8.8,24.0,0.368,25.5,49.1,0.519,15.6,20.2,0.774,9.8,28.8,38.7,21.1,5.8,3.2,12.5,15.9,93.1,2016,NBA,0.653


### Combining the NBA and EL Teams' DataFrames

Now we can finally create a single DataFrame containing the information for all teams from both leagues.

In [92]:
# Combine the two teams DataFrame into a single table
teams = pd.concat([NBA_teams_clean, EL_teams_clean], ignore_index = True)

In [93]:
# Look at the final result
teams.info()
teams

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Team    234 non-null    object 
 1   GP      234 non-null    int64  
 2   FGM     234 non-null    float64
 3   FGA     234 non-null    float64
 4   FG%     234 non-null    float64
 5   3PM     234 non-null    float64
 6   3PA     234 non-null    float64
 7   3P%     234 non-null    float64
 8   2PM     234 non-null    float64
 9   2PA     234 non-null    float64
 10  2P%     234 non-null    float64
 11  FTM     234 non-null    float64
 12  FTA     234 non-null    float64
 13  FT%     234 non-null    float64
 14  ORB     234 non-null    float64
 15  DRB     234 non-null    float64
 16  TRB     234 non-null    float64
 17  AST     234 non-null    float64
 18  STL     234 non-null    float64
 19  BLK     234 non-null    float64
 20  TOV     234 non-null    float64
 21  PF      234 non-null    float64
 22  PT

,Team,GP,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,League,W/L%
0,Golden State Warriors,82,35.9,72.6,0.495,10.0,26.0,0.383,25.9,46.5,0.557,14.8,18.8,0.788,7.8,29.2,37.0,25.3,8.0,5.7,12.3,16.1,96.6,2016,NBA,0.542
1,Houston Rockets,82,33.6,72.7,0.462,12.0,33.6,0.357,21.6,39.1,0.552,16.9,22.1,0.766,9.1,27.9,37.0,21.0,6.8,3.6,12.6,16.6,96.1,2016,NBA,0.236
2,Denver Nuggets,82,34.3,73.1,0.469,8.8,24.0,0.368,25.5,49.1,0.519,15.6,20.2,0.774,9.8,28.8,38.7,21.1,5.8,3.2,12.5,15.9,93.1,2016,NBA,0.653
3,Cleveland Cavaliers,82,33.2,70.8,0.470,10.8,28.2,0.384,22.4,42.5,0.528,14.6,19.4,0.748,7.8,28.7,36.4,18.9,5.5,3.3,11.4,15.1,91.9,2016,NBA,0.306
4,Washington Wizards,82,34.4,72.5,0.475,7.7,20.7,0.372,26.8,51.9,0.515,14.4,18.4,0.784,8.6,27.2,35.8,19.9,7.1,3.4,11.8,17.8,91.0,2016,NBA,0.472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Khimki,34,27.5,62.2,0.442,9.3,26.8,0.346,18.2,35.4,0.514,13.2,16.3,0.810,7.5,20.8,28.3,18.2,7.0,3.3,13.9,22.1,77.5,2020,EL,0.118
230,Zenit Saint Petersburg,39,27.2,56.7,0.480,9.2,24.5,0.375,18.0,32.2,0.559,13.8,17.4,0.792,7.7,20.5,28.1,17.5,6.0,2.0,12.2,20.9,77.5,2020,EL,0.588
231,Zalgiris,34,28.4,58.5,0.486,8.4,20.0,0.421,20.0,38.5,0.519,11.9,14.1,0.840,8.1,20.7,28.8,18.3,6.4,1.1,13.3,19.4,77.4,2020,EL,0.500
232,ASVEL Basket,34,27.4,57.9,0.474,8.4,21.4,0.393,19.0,36.5,0.521,13.3,17.2,0.774,7.3,20.8,28.1,16.4,6.6,3.2,14.5,21.5,76.7,2020,EL,0.382


In [94]:
# Save the DataFrame to a CSV file
teams.to_csv("final_DataFrames/team_stats_clean.csv")